# Clone Repo




In [ ]:
!git clone https://github.com/ShesterG/AfriSign.git

Cloning into 'AfriSign'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 44 (delta 7), reused 8 (delta 2), pack-reused 0
Unpacking objects: 100% (44/44), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
from pathlib import Path

In [ ]:
%cd AfriSign/Datasets

/content/AfriSign/Datasets


#Install Requirements

In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Download videos

In [ ]:
#edit line13 of downloader, lan --> ase
!python video_downloader.py --save_path /content/drive/MyDrive/Sign_Language_Videos/ase

#Save infos about the video

In [ ]:
!python videos_info.py --save_path /content/drive/MyDrive/Sign_Language_Videos/ --sign_lang ase

# Convert videos to tensors

In [ ]:
!python videos_to_tensors.py --save_path /content/drive/MyDrive/Sign_Language_Videos/ase

#Verse Text

In [ ]:
import pandas as pd
import numpy as np
import math
import sys
import pickle
import gzip
!pip install requests
import requests
from google.colab import files
from pathlib import 
from random import shuffle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [ ]:
def getBibleText(verse_name):
  baseUrl = f'https://bible-api.com/{verse_name}'
  res = requests.get(baseUrl)
  json = res.json()
  return json['text']

In [ ]:
lan = "ase"
data_list1 = load_dataset_file(f"/content/drive/MyDrive/Sign_Language_Videos/dataset/{lan}1.dataset")
data_list2 = load_dataset_file(f"/content/drive/MyDrive/Sign_Language_Videos/dataset/{lan}2.dataset")
data_list3 = load_dataset_file(f"/content/drive/MyDrive/Sign_Language_Videos/dataset/{lan}3.dataset")
data_list4 = load_dataset_file(f"/content/drive/MyDrive/Sign_Language_Videos/dataset/{lan}4.dataset")
data_list5 = load_dataset_file(f"/content/drive/MyDrive/Sign_Language_Videos/dataset/{lan}5.dataset")
data_list6 = load_dataset_file(f"/content/drive/MyDrive/Sign_Language_Videos/dataset/{lan}6.dataset")
data_list7 = load_dataset_file(f"/content/drive/MyDrive/Sign_Language_Videos/dataset/{lan}7.dataset")
data_list8 = load_dataset_file(f"/content/drive/MyDrive/Sign_Language_Videos/dataset/{lan}8.dataset")
data_list = data_list1 + data_list2 + data_list3 + data_list4 + data_list5 + data_list6 + data_list7 + data_list8

In [ ]:
data_list = load_dataset_file(f"/content/drive/MyDrive/Sign_Language_Videos/dataset/T{lan}240.dataset")

In [ ]:
len(data_list)

In [ ]:
verses = []
unique_data_list = []
for i in range(len(data_list)):
  if data_list[i]["name"] in verses:
    continue
  else:
    unique_data_list.append(data_list[i])
    verses.append(data_list[i]["name"])

In [ ]:
data_list = []
data_list = unique_data_list

In [ ]:
len(data_list)

In [ ]:
for i in range(len(data_list)):
  #data_list[i]["sign"] = data_list[i]["sign"].to(dtype=torch.float32)
    try:
      data_list[i]["text"] = getBibleText(data_list[i]["name"])
    except KeyError:
      print(i)

In [ ]:
unique_data_list = []
for i in range(len(data_list)):
  if data_list[i]["text"]=="Text":
    continue
  else:
    unique_data_list.append(data_list[i])

In [ ]:
data_list = []
data_list = unique_data_list

In [ ]:
punctuation_list = ['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~',
 '“',
 '”',
 '‘',
 '‘',
 '.',
 '?',
 '—']

In [ ]:
for i in range(len(data_list)):
  data_list[i]["text"] = data_list[i]["text"].lower().translate({ord(x): ' ' for x in punctuation_list}).replace('\n', ' ').replace('’', '').replace('       ', ' ').replace('      ', ' ').replace('     ', ' ').replace('    ', ' ').replace('   ', ' ').replace('  ', ' ').strip() + " ."

#Create vocab file

In [ ]:
voc = set("<unk>","<pad>","<s>","</s>")
for i in range(len(data_list)):
  verse_words = data_list[i]['text'].split()
  for vw in verse_words:
    voc.add(vw) 

In [ ]:
##to save in file
with open(f'/content/drive/MyDrive/Sign_Language_Videos/dataset/{language}240/txt.vocab', 'w') as f:
    for line in voc:
        f.write(f"{line}\n")

In [ ]:
voclist = []
for i in range(len(data_list)):
  verse_words = data_list[i]['text'].split()
  for vw in verse_words:
    voclist.append(vw)

In [ ]:
print(f"length of vocabulary is {len(voc)}")
print(f"total number of words in dataset is {len(voclist)}")

#Split into train, valid and test sets

In [ ]:
data_list[0]

In [ ]:
len(data_list)

In [ ]:
shuffle(data_list)

In [ ]:
training = data_list[:2800] 
validation = data_list[2800:3100] 
testing = data_list[3100:] 

In [ ]:
print(len(training))
print(len(validation))
print(len(testing))

In [ ]:
def save_list(object, filename, protocol = 0):
        """Saves a compressed object to disk
        """
        file = gzip.GzipFile(filename, 'wb')
        file.write(pickle.dumps(object, protocol))
        file.close()

In [ ]:
language = lan.upper()

In [ ]:
save_list(training,f"/content/drive/MyDrive/Sign_Language_Videos/dataset/{language}240/{language}240.train")
save_list(validation,f"/content/drive/MyDrive/Sign_Language_Videos/dataset/{language}240/{language}240.dev")
save_list(testing,f"/content/drive/MyDrive/Sign_Language_Videos/dataset/{language}240/{language}240.test")